In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
join_session = SparkSession.builder.appName("JoinExample").getOrCreate()

25/11/30 08:45:57 WARN Utils: Your hostname, Shithils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/11/30 08:45:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/30 08:45:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/30 08:45:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_spark=join_session.read.option("header","true").option("inferSchema","true").csv("train.csv")
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



In [4]:
store_info = join_session.createDataFrame([
    (28, "Rural"),
    (5, "Urban"),
    (17, "Suburban")
], ["Store", "StoreType"])
store_info.show()


+-----+---------+
|Store|StoreType|
+-----+---------+
|   28|    Rural|
|    5|    Urban|
|   17| Suburban|
+-----+---------+



In [5]:
df_spark.join(store_info, on="Store", how="inner").show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|StoreType|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|   17|        5|2015-07-31| 8430|      946|   1|    1|           0|            1| Suburban|
|   17|        4|2015-07-30| 7555|      850|   1|    1|           0|            1| Suburban|
|   17|        3|2015-07-29| 7179|      861|   1|    1|           0|            1| Suburban|
|   17|        2|2015-07-28| 9316|      983|   1|    1|           0|            1| Suburban|
|   17|        1|2015-07-27| 8293|      904|   1|    1|           0|            1| Suburban|
|   17|        7|2015-07-26|    0|        0|   0|    0|           0|            0| Suburban|
|   17|        6|2015-07-25| 2572|      357|   1|    0|           0|            0| Suburban|
|   17|        5|2015-07-24| 4443|      657|   1|    0|           0|  

In [6]:
df_spark=df_spark.join(store_info, on="Store", how="left")
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|StoreType|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|    7|        5|2015-07-31|15344|     1414|   1|    1|           0|            1|     NULL|
|    6|        5|2015-07-31| 5651|      589|   1|    1|           0|            1|     NULL|
|    9|        5|2015-07-31| 8565|      687|   1|    1|           0|            1|     NULL|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|    Urban|
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|     NULL|
|   10|        5|2015-07-31| 7185|      681|   1|    1|           0|            1|     NULL|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|     NULL|
|    8|        5|2015-07-31| 8492|      833|   1|    1|           0|  

In [7]:
df_spark=df_spark.join(df_spark, on="Store", how="inner")
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|StoreType|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|StoreType|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|   28|        5|2015-07-31| 7301|      663|   1|    1|           0|            1|    Rural|        5|2015-07-31| 7301|      663|   1|    1|           0|            1|    Rural|
|   28|        5|2015-07-31| 7301|      663|   1|    1|           0|            1|    Rural|        4|2015-07-30| 6210|      635|   1|    1|           0|            1|    Rural|
|   28|        5|2015-07-31| 7301|      663|   1|    1|           0|            1|    Rural|        3|2015-07-

Broadcast Join

In [8]:
broad_df=join_session.read.option("header","true").option("inferSchema","true").csv("train.csv")
broad_df.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



What Spark does here?
- It takes store_info
- Compresses it
- Sends a copy to every executor
-Then executors join locally with their partition of sales_df
No shuffle, no network overhead → much faster.

In [10]:
result_df=broad_df.join(broadcast(store_info), on="Store", how="inner")
result_df.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|StoreType|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+---------+
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|    Urban|
|   17|        5|2015-07-31| 8430|      946|   1|    1|           0|            1| Suburban|
|   28|        5|2015-07-31| 7301|      663|   1|    1|           0|            1|    Rural|
|    5|        4|2015-07-30| 4943|      539|   1|    1|           0|            1|    Urban|
|   17|        4|2015-07-30| 7555|      850|   1|    1|           0|            1| Suburban|
|   28|        4|2015-07-30| 6210|      635|   1|    1|           0|            1|    Rural|
|    5|        3|2015-07-29| 5899|      658|   1|    1|           0|            1|    Urban|
|   17|        3|2015-07-29| 7179|      861|   1|    1|           0|  